In [1]:
from MS461xx_RawSockets_example import * # from https://github.com/Anritsu/Examples/blob/main/BenchtopVNAs/Python/MS461xx_RawSockets_example.py
#https://dl.cdn-anritsu.com/en-us/test-measurement/files/Manuals/Programming-Manual/10410-00333H.pdf
import numpy as np
import skrf as rf
import json
import time
from datetime import datetime

In [2]:
vna = InstrumentSocketConnection()

In [4]:
vna.query('*IDN?')

'ANRITSU,MS46122B,2250307,V2025.4.3,V2.3.20'

In [ ]:
traceindex= 0
while True:
    file_object = open("vna-control.json", "r")
    try:
        vnastate = json.loads(file_object.read())
    except ValueError:
        pass
    vna.write(":SENS1:FREQ:START " + str(vnastate['start']))
    vna.write(":SENS1:FREQ:STOP " + str(vnastate['stop']))
    vna.write(":SENSe1:SWEep:POINt " + str(vnastate['numpoints']))
    vna.write(":SENS1:BAND 1000" + str(vnastate['ifbw']))
    vna.write(":SOUR1:POW " + vnastate['power'])
    fraw = vna.query(":SENS1:FREQ:DATA?")
    fstringarray = fraw[12:].split('\n')
    flist = []
    for fstring in fstringarray:
        flist.append(float(fstring))
    f = np.array(flist)
    
    vna.write(":TRIG:SING")
    opc_done = vna.query("*OPC?")
    if not (opc_done == "1"):
        print(f"Error, Opc returned unexpected value while waiting for a single sweep to finish (expected '1', received {opc_done}); ending code execution.")
        exit()
    
    s11string = vna.query(":CALC1:PAR1:DATA:SDAT?")  # Reading S-data from trace 1 - S11
    s21string = vna.query(":CALC1:PAR2:DATA:SDAT?")  # Reading S-data from trace 2 - S21
    s12string = vna.query(":CALC1:PAR3:DATA:SDAT?")  # Reading S-data from trace 3 - S12
    s22string = vna.query(":CALC1:PAR4:DATA:SDAT?")  # Reading S-data from trace 4 - S22
    s11stringarray = s11string[12:].split("\n")
    s11 = []
    for s in s11stringarray:
        real = float(s.split(",")[0])
        imag = float(s.split(",")[1])
        sval = complex(real,imag)
        s11.append(sval)
    s11 = np.array(s11)
    
    s21stringarray = s21string[12:].split("\n")
    s21 = []
    for s in s21stringarray:
        real = float(s.split(",")[0])
        imag = float(s.split(",")[1])
        sval = complex(real,imag)
        s21.append(sval)
    s21 = np.array(s21)
    
    s12stringarray = s12string[12:].split("\n")
    s12 = []
    for s in s12stringarray:
        real = float(s.split(",")[0])
        imag = float(s.split(",")[1])
        sval = complex(real,imag)
        s12.append(sval)
    s12 = np.array(s12)
    
    s22stringarray = s22string[12:].split("\n")
    s22 = []
    for s in s22stringarray:
        real = float(s.split(",")[0])
        imag = float(s.split(",")[1])
        sval = complex(real,imag)
        s22.append(sval)
    s22 = np.array(s22)
    
    s = []
    for index in range(len(s11)):
        s0 = np.array([[0+0j,0+0j],[0+0j,0+0j]])
        s0[0,0] = s11[index]
        s0[0,1] = s12[index]
        s0[1,0] = s21[index]
        s0[1,1] = s22[index]    
        s.append(s0)
    s = np.array(s)
    trace = rf.Network(frequency=f, s=s, name='data trace')
    
    # apply calibration here from ecal if applicable
    
    trace.write_touchstone('trace.s2p')
   # file_object = open("state-indicator.json", "w")
   # file_object.write(json.dumps(vnastate))
   # file_object.close()
    
    tracejson = {}
    tracejson['s11db'] = trace.s11.s_db[:,0,0].tolist()
    tracejson['s12db'] = trace.s12.s_db[:,0,0].tolist()
    tracejson['s21db'] = trace.s21.s_db[:,0,0].tolist()
    tracejson['s22db'] = trace.s22.s_db[:,0,0].tolist()
    tracejson['s11deg'] = trace.s11.s_deg[:,0,0].tolist()
    tracejson['s12deg'] = trace.s12.s_deg[:,0,0].tolist()
    tracejson['s21deg'] = trace.s21.s_deg[:,0,0].tolist()
    tracejson['s22deg'] = trace.s22.s_deg[:,0,0].tolist()
    tracejson['f'] = trace.f.tolist()
    
    file_object = open("trace.json", "w")
    file_object.write(json.dumps(tracejson))
    file_object.close()

    nowjson = {}
    nowjson['traceindex'] = traceindex
    nowjson['datetime'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    file_object = open("now.json", "w")
    file_object.write(json.dumps(nowjson))
    file_object.close()

    traceindex = traceindex + 1